In [1]:
import pandas as pd
import numpy as np
import re
import pytz
import os
from pathlib import Path
import sys
sys.path.append("/home/jovyan/shared/service-data")

from src.clean import clean_percentage, normalize_string, standardize_column_names, clean_fiscal_yr
from src.load import load_csv_from_raw
from src.export import export_to_csv
from src.merge import merge_si, merge_ss

In [39]:
base_dir = Path.cwd()
parent_dir = base_dir.parent

ifoi_en = pd.read_csv(parent_dir / "inputs" / "ifoi_en.csv")
ifoi_fr = pd.read_csv(parent_dir / "inputs" / "ifoi_fr.csv")

ifoi_en = ifoi_en.set_index(ifoi_en.columns[0], drop=True).add_suffix('_en')
ifoi_fr = ifoi_fr.set_index(ifoi_fr.columns[0], drop=True).add_suffix('_fr')

ifoi_en.set_index(ifoi_en.columns[0], drop=True)
ifoi_fr.set_index(ifoi_fr.columns[0], drop=True)

ifoi = pd.concat([ifoi_en, ifoi_fr], axis=1)

# Extract column lists
en_cols = ifoi_en.columns.tolist()
fr_cols = ifoi_fr.columns.tolist()

# Interleave them by index
merged_cols = [col for pair in zip(en_cols, fr_cols) for col in pair]

# Apply new column order
ifoi = standardize_column_names(ifoi[merged_cols].reset_index())

ifoi

,org_id,legal_title_en,appellation_légale_fr,applied_title_en,titre_appliqué_fr,website_en,site_web_fr,enabling_instruments_en,instruments_habilitants_fr,description_en,...,auditor_en,vérificateur_fr,federal_ownership_or_representation_en,participation_fédérale_ou_représentants_fr,status_en,statut_fr,end_date_en,date_de_fin_fr,notes_en,notes_fr
0,1,Department of Agriculture and Agri-Food,Ministère de l’Agriculture et de l’Agroaliment...,Agriculture and Agri-Food Canada,Agriculture et Agroalimentaire Canada,www.agr.gc.ca,www.agr.gc.ca,"Department of Agriculture and Agri-Food Act, R...",Loi sur le ministère de l’Agriculture et de l’...,The Department of Agriculture and Agri-Food (A...,...,NaN,NaN,NaN,NaN,a,a,NaN,NaN,NaN,NaN
1,2,House of Commons,Chambre des communes,NaN,NaN,www.parl.gc.ca,www.parl.gc.ca,"Parliament of Canada Act, R.S.C., 1985, c. P-1","Loi sur le Parlement du Canada, L.R.C., 1985, ...",The House of Commons is where Members of Parli...,...,NaN,NaN,NaN,NaN,a,a,NaN,NaN,NaN,NaN
2,4,Aboriginal Healing Foundation,Fondation autochtone de guérison,NaN,NaN,NaN,NaN,Incorporated through the Funding Agreements wi...,Incorporée à la suite d’accords de financement...,The Foundation was created in response to the ...,...,NaN,NaN,NaN,NaN,d,d,2013.0,2013.0,NaN,NaN
3,5,Aéroport de Québec Inc.,Aéroport de Québec Inc.,NaN,NaN,www.aeroportdequebec.com,www.aeroportdequebec.com,Canada Not-for-profit Corporations Act (S.C. 2...,Loi canadienne sur les organisations à but non...,"To manage, operate, and develop the airport.",...,Raymond Chabot Grant Thornton LLP,Raymond Chabot Grant Thornton S.E.N.C.R.L.,2/15 board members,Deux membres sur quinze,a,a,NaN,NaN,NaN,NaN
4,6,Aéroports de Montréal,Aéroports de Montréal,NaN,NaN,www.admtl.com,www.admtl.com,Canada Not-for-profit Corporations Act (S.C. 2...,Loi canadienne sur les organisations à but non...,"To manage, operate, and develop the airports.",...,KPMG LLP,KPMG s.r.l.,2/15 board members,Deux membres sur quinze,a,a,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,559,Healthcare Excellence Canada (HEC),Excellence en santé Canada (ESC),NaN,NaN,https://www.healthcareexcellence.ca/en/,https://www.healthcareexcellence.ca/fr/,NaN,NaN,Healthcare Excellence Canada was launched in 2...,...,Ernst & Young,Ernst & Young,1/12 board members,1/12 du conseil d’administration,a,a,NaN,NaN,NaN,NaN
323,560,Pacific Economic Development Agency of Canada,Agence de développement économique du Pacifiqu...,NaN,NaN,https://www.canada.ca/en/pacific-economic-deve...,https://www.canada.ca/fr/developpement-economi...,PC-2021-0827,CP-2021-0827,Pacific Economic Development Canada (PacifiCan...,...,NaN,NaN,NaN,NaN,a,a,NaN,NaN,Her Excellency the Governor General in Council...,"Sur recommandation du premier ministre, Son Ex..."
324,561,Federal Economic Development Agency for Northe...,Agence fédérale de développement économique po...,NaN,NaN,http://fednor.gc.ca/eic/site/fednor-fednor.nsf...,http://fednor.gc.ca/eic/site/fednor-fednor.nsf...,PC-2021-0839,CP-2021-0839,FedNor is the Government of Canada's economic ...,...,NaN,NaN,NaN,NaN,a,a,NaN,NaN,Her Excellency the Governor General in Council...,"Sur recommandation du premier ministre, Son Ex..."
325,562,VIA HFR - VIA TGF Inc.,VIA HFR - VIA TGF Inc.,NaN,NaN,https://hfr-tgf.ca/,https://tgf-hfr.ca/,"Canada Business Corporations Act, R.S.C. 1985,...","Loi canadienne sur les sociétés par actions, L...",VIA HFR – VIA TGF Inc. (VIA HFR) was establish...,...,NaN,NaN,NaN,NaN,a,a,NaN,NaN,NaN,NaN
